In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm
import pandas as pd
from data_sources import process_parallel, transform_csv_row

In [4]:
from collections import Counter

In [5]:
import scipy.sparse

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [6]:
version = 'v6'
data_path = Path('data/midi')
version_path = data_path/version

In [7]:
source_dir = 'midi_transform'
out_dir = 'midi_npz'
source_csv = version_path/source_dir/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'{out_dir}.csv'

In [8]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,9,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,inferred_key,inferred_offset,genres,section,parts,ht_time_signature,ht_bpm,midi_title,mxl,midi_transform
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,C major,0.0,NaN,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,C major,0.0,NaN,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,A minor,0.0,Jazz,chorus,chorus,4.0,96.0,kiefer,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,D minor,-5.0,"J-Pop,Pop",verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN,midi_transform/hooktheory/pianoroll/w/whitefla...
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,C# major,-1.0,Holiday,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...


In [9]:
df_filtered = df.loc[df[source_dir].notna()]; df_filtered.shape

(38837, 23)

In [10]:
def transform_func(file, out_file, row):
    stream = file2stream(file) # 1.
    chordarr = stream2chordarr(stream) # 2.
    if row.source != 'hooktheory': # keep hooktheory
        chord_short = compress_chordarr(chordarr)
        delta_trim = chordarr.shape[0] - chord_short.shape[0]
        if delta_trim > 100: print(f'Removed {delta_trim} rests from {file}')
        chordarr = chord_short
    save_chordarr(out_file, chordarr)

In [11]:
from functools import partial
parallel_func = partial(transform_csv_row, 
        transform_func=transform_func,
        base_path=version_path,
        source_dir=source_dir,
        out_dir=out_dir,
        out_extension='.npz'
       )

In [12]:
# for r in df_filtered.iterrows():
#     parallel_func(r)
#     break

In [13]:
transcribed_files = process_parallel(parallel_func, df_filtered.iterrows(), total=df_filtered.shape[0])

Compressing rests: 44 -> 32
Compressing rests: 44 -> 32
Compressing rests: 82 -> 34
Compressing rests: 56 -> 32
Compressing rests: 44 -> 32
Compressing rests: 44 -> 32
Compressing rests: 82 -> 34
Compressing rests: 177 -> 33
Removed 144 rests from data/midi/v6/midi_transform/freemidi/genre-dance-eletric/Depeche Mode - I Sometimes Wish That I Was Dead.mid
Compressing rests: 51 -> 35
Compressing rests: 42 -> 34
Compressing rests: 84 -> 32
Compressing rests: 80 -> 32
Removed 156 rests from data/midi/v6/midi_transform/freemidi/genre-dance-eletric/Radiohead - 15 Step.mid
Compressing rests: 58 -> 34
Removed 128 rests from data/midi/v6/midi_transform/freemidi/genre-pop/Corrs - No Good For Me.mid
Compressing rests: 386 -> 34
Removed 352 rests from data/midi/v6/midi_transform/freemidi/genre-dance-eletric/Garbage - Hammering In My Head.mid
Compressing rests: 60 -> 32
Compressing rests: 128 -> 32
Removed 124 rests from data/midi/v6/midi_transform/freemidi/genre-dance-eletric/Breathe Carolina - No

Compressing rests: 84 -> 32
Compressing rests: 72 -> 32
Removed 1260 rests from data/midi/v6/midi_transform/from_mxl/wikifonia/Luis Miguel - Mexico En La Piel.mid
Error converting midi to sequence index 127 is out of bounds for axis 2 with size 127
Compressing rests: 40 -> 32
Compressing rests: 168 -> 32
Compressing rests: 182 -> 34
Compressing rests: 262 -> 34
Compressing rests: 248 -> 32
Compressing rests: 194 -> 34
Compressing rests: 126 -> 34
Compressing rests: 126 -> 34
Compressing rests: 84 -> 32
Compressing rests: 72 -> 32
Removed 1260 rests from data/midi/v6/midi_transform/from_mxl/wikifonia/Luis Miguel - Mexico En La Piel (Trumpet in Bb).mid
Compressing rests: 60 -> 32
Compressing rests: 56 -> 32
Compressing rests: 44 -> 32
Compressing rests: 52 -> 32
Compressing rests: 260 -> 32
Removed 388 rests from data/midi/v6/midi_transform/from_mxl/wikifonia/Bon Jovi - Always.mid
Removed 128 rests from data/midi/v6/midi_transform/from_mxl/wikifonia/Stuart Duaermann, Steffi Geiser Rubin 

In [14]:
tdf = pd.DataFrame(data={out_dir: list(transcribed_files.values())}, index=list(transcribed_files.keys()))

In [15]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

((38837, 1), (39942, 23), (39942, 24))

In [16]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,inferred_offset,genres,section,parts,ht_time_signature,ht_bpm,midi_title,mxl,midi_transform,midi_npz
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,0.0,NaN,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,0.0,NaN,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,0.0,Jazz,chorus,chorus,4.0,96.0,kiefer,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,-5.0,"J-Pop,Pop",verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN,midi_transform/hooktheory/pianoroll/w/whitefla...,midi_npz/hooktheory/pianoroll/w/whiteflame/sen...
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,-1.0,Holiday,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...,midi_npz/hooktheory/pianoroll/w/wham/last-chri...


### Calculate timesteps

In [17]:
merged_df = pd.read_csv(out_csv)

In [18]:
def calc_timesteps(idxrow):
    idx,row = idxrow
    np_file = row[out_dir]
    if not isinstance(np_file, str) or not (Path(version_path)/np_file).exists(): return idx,None
    try:
#         timesteps = scipy.sparse.load_npz(np_file).shape[0]
        timesteps = load_chordarr(Path(version_path)/np_file).shape[0]
        return idx,timesteps
    except Exception as e:
        print('Error reading text', e, np_file)
    return idx, None

In [19]:
# for r in merged_df.iterrows():
#     calc_timesteps(r)

In [20]:
file2steps = process_parallel(calc_timesteps, merged_df.iterrows(), total=merged_df.shape[0])

In [21]:
len_df = pd.DataFrame(data={f'{out_dir}_timesteps': list(file2steps.values())}, index=list(file2steps.keys()))
merged_len_df = merged_df.join(len_df, how='outer');
len_df.shape, merged_df.shape, merged_len_df.shape

((39942, 1), (39942, 24), (39942, 25))

In [22]:
merged_len_df.to_csv(out_csv, index=False); merged_len_df.head()

,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,genres,section,parts,ht_time_signature,ht_bpm,midi_title,mxl,midi_transform,midi_npz,midi_npz_timesteps
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,NaN,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,NaN,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,Jazz,chorus,chorus,4.0,96.0,kiefer,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,"J-Pop,Pop",verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN,midi_transform/hooktheory/pianoroll/w/whitefla...,midi_npz/hooktheory/pianoroll/w/whiteflame/sen...,129.0
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,Holiday,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...,midi_npz/hooktheory/pianoroll/w/wham/last-chri...,129.0
